# SQL 的五十道練習

> 更多綜合練習題

[數據交點](https://www.datainpoint.com) | 郭耀仁 <yaojenkuo@datainpoint.com>

## 練習題指引

- 由於近期 mybinder.org 的服務不穩定，新增 Google Colab 作為另一個寫作練習題的平台。
- 開始寫作之前，可以先按上方「Copy to Drive」按鈕將筆記本複製到自己的 Google 雲端硬碟。
- 在每份練習題的開始，都會先將四個學習資料庫載入環境。
- 因此 SQL 可以指定四個學習資料庫中的資料表，不需要額外指定資料庫。
- 在 SQL 語法起點與 SQL 語法終點這兩個單行註解之間撰寫能夠得到預期結果的 SQL。
- 可以先在自己電腦的 SQLiteStudio 或者 DBeaver 寫出跟預期結果相同的 SQL 後再複製貼上到練習題。
- 執行測試的方式為點選右上角 Connect，然後點選上方選單的 Runtime -> Restart and run all -> Yes -> Run anyway。
- 移動到 Google Colab 的最後一個儲存格看測試結果。
- 可以每寫一題就執行測試，也可以全部寫完再執行測試。
- 練習題閒置超過 10 分鐘會自動斷線，這時只要重新點選練習題連結即可重新啟動。

In [79]:
!wget -N https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/nba.db
!wget -N https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/covid19.db
!wget -N https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/twElection2020.db
!wget -N https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/imdb.db
!wget -N https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/exercise_index.json

--2026-01-20 08:31:40--  https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/nba.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233472 (228K) [application/octet-stream]
Saving to: ‘nba.db’

nba.db              100%[===================>] 228.00K  --.-KB/s    in 0.03s   

Last-modified header missing -- time-stamps turned off.
2026-01-20 08:31:40 (7.75 MB/s) - ‘nba.db’ saved [233472/233472]

--2026-01-20 08:31:40--  https://raw.githubusercontent.com/datainpoint/classroom-hahow-sqlfifty/main/databases/covid19.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... c

In [80]:
import sqlite3
import unittest
import json
import os
import numpy as np
import pandas as pd
conn = sqlite3.connect('nba.db')
conn.execute("""ATTACH 'covid19.db' AS covid19""")
conn.execute("""ATTACH 'twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH 'imdb.db' AS imdb""")

## 51. 從 `covid19` 資料庫查詢截至 2021-03-31 美國前十大確診人數的州別，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(10, 2) 的查詢結果。

```
   Province_State  Confirmed
0      California    3668277
1           Texas    2791910
2         Florida    2057735
3        New York    1873138
4        Illinois    1244499
5         Georgia    1059548
6    Pennsylvania    1027678
7            Ohio    1017566
8  North Carolina     914132
9      New Jersey     908816
```

In [81]:
summarize_confirmed_by_us_states_from_covid19 =\
"""
-- SQL 查詢語法起點
SELECT lookup_table.Province_State,
       SUM(Confirmed) AS Confirmed
  FROM daily_report
  JOIN lookup_table
    ON daily_report.Combined_Key = lookup_table.Combined_Key
WHERE lookup_table.Country_Region = 'US'
GROUP BY lookup_table.Province_State
ORDER BY Confirmed DESC
LIMIT 10;
-- SQL 查詢語法終點
"""

## 52. 從 `covid19` 資料庫查詢截至 2021-03-31 台灣、日本、中國、南韓與新加坡五個國家的確診與死亡人數的資訊，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(5, 3) 的查詢結果。

```
  Country_Region  Confirmed  Deaths
0          China     101732    4841
1          Japan     474641    9155
2   Korea, South     103639    1735
3      Singapore      60381      30
4         Taiwan       1030      10
```

In [82]:
summarize_confirmed_deaths_of_east_asia_from_covid19 =\
"""
-- SQL 查詢語法起點
SELECT lookup_table.Country_Region,
       SUM(daily_report.Confirmed) AS Confirmed,
       SUM(daily_report.Deaths) AS Deaths
  FROM daily_report
  JOIN lookup_table
    ON daily_report.Combined_Key = lookup_table.Combined_Key
WHERE lookup_table.Country_Region IN ('China','Japan','Korea, South','Singapore','Taiwan')
GROUP BY lookup_table.Country_Region;
-- SQL 查詢語法終點
"""

## 53. 從 `imdb` 資料庫查詢出現最多次的導演為誰，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(3, 2) 的查詢結果。

```
            director  counts
0  Christopher Nolan       7
1    Martin Scorsese       7
2    Stanley Kubrick       7
```

In [83]:
find_most_frequent_directors_from_imdb =\
"""
-- SQL 查詢語法起點
SELECT director,
       counts
  FROM (
           SELECT director,
                  COUNT( * ) AS counts
             FROM movies
            GROUP BY director
       )
       AS counts_by_directors
 WHERE counts = (
                    SELECT MAX(counts)
                      FROM (
                               SELECT director,
                                      COUNT( * ) AS counts
                                 FROM movies
                                GROUP BY director
                           )
                           AS counts_by_directors
                );
-- SQL 查詢語法終點
"""

## 54. 從 `imdb` 資料庫查詢出現最多次的演員為誰，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(3, 2) 的查詢結果。

```
   actor_id            name  counts
0      2472  Robert De Niro       9
```

In [84]:
find_most_frequent_actors_from_imdb =\
"""
-- SQL 查詢語法起點
SELECT counts_by_actors.actor_id,
       actors.name,
       counts_by_actors.counts
  FROM (SELECT actor_id,
       COUNT(*) AS counts
       FROM casting
      GROUP BY actor_id) AS counts_by_actors
  JOIN actors
    ON counts_by_actors.actor_id = actors.id
WHERE counts = (SELECT MAX(counts)
  FROM (SELECT actor_id,
       COUNT(*) AS counts
  FROM casting
GROUP BY actor_id) AS counts_by_actors);
-- SQL 查詢語法終點
"""

## 55. 從 `imdb` 資料庫查詢評等大於等於 8.8（`rating >= 8.8`）電影的導演以及第一主角（`ord = 1`），參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(13, 3) 的查詢結果。

```
                                                title              director  \
0                            The Shawshank Redemption        Frank Darabont   
1                                       The Godfather  Francis Ford Coppola   
2                              The Godfather: Part II  Francis Ford Coppola   
3                                     The Dark Knight     Christopher Nolan   
4                                        12 Angry Men          Sidney Lumet   
5                                    Schindler's List      Steven Spielberg   
6       The Lord of the Rings: The Return of the King         Peter Jackson   
7                                        Pulp Fiction     Quentin Tarantino   
8                      The Good, the Bad and the Ugly          Sergio Leone   
9   The Lord of the Rings: The Fellowship of the Ring         Peter Jackson   
10                                         Fight Club         David Fincher   
11                                       Forrest Gump       Robert Zemeckis   
12                                          Inception     Christopher Nolan   

           lead_actor  
0         Tim Robbins  
1       Marlon Brando  
2           Al Pacino  
3      Christian Bale  
4       Martin Balsam  
5         Liam Neeson  
6        Noel Appleby  
7            Tim Roth  
8         Eli Wallach  
9         Alan Howard  
10      Edward Norton  
11          Tom Hanks  
12  Leonardo DiCaprio  
```

In [85]:
find_lead_actors_from_imdb =\
"""
-- SQL 查詢語法起點
SELECT movies.title,
       movies.rating,
       actors.name AS lead_actor
  FROM movies
  JOIN casting
    ON movies.id = casting.movie_id
  JOIN actors
    ON casting.actor_id = actors.id
WHERE movies.rating >=8.8 AND
      casting.ord = 1;
-- SQL 查詢語法終點
"""

## 56. 從 `nba` 資料庫查詢截至 2021-03-31 的得分王（生涯總得分 `points` 最高）、助攻王（生涯總助攻 `assists` 最高）、籃板王（生涯總籃板 `totReb` 最高）、抄截王（生涯總抄截 `steals` 最高）以及阻攻王（生涯總阻攻 `blocks` 最高），參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(5, 4) 的查詢結果。

```
  firstName lastName category  value
0     Chris     Paul  assists  10051
1     Chris     Paul   steals   2296
2    Dwight   Howard   blocks   2173
3    Dwight   Howard   totReb  14071
4    LeBron    James   points  35283
```

In [86]:
find_max_stats_total_from_nba =\
"""
-- SQL 查詢語法起點
SELECT players.firstname,
       players.lastname,
       'points' AS category,
       career_summaries.points AS value
  FROM players
       JOIN
       career_summaries ON players.personId = career_summaries.personId
 WHERE career_summaries.personID = (
                                       SELECT personID
                                         FROM career_summaries
                                        WHERE points = (
                                                           SELECT MAX(points)
                                                             FROM career_summaries
                                                       )
                                   )
UNION
SELECT players.firstname,
       players.lastname,
       'totReb' AS category,
       career_summaries.totReb AS value
  FROM players
       JOIN
       career_summaries ON players.personId = career_summaries.personId
 WHERE career_summaries.personID = (
                                       SELECT personID
                                         FROM career_summaries
                                        WHERE totReb = (
                                                           SELECT MAX(totReb)
                                                             FROM career_summaries
                                                       )
                                   )
UNION
SELECT players.firstname,
       players.lastname,
       'blocks' AS category,
       career_summaries.blocks AS value
  FROM players
       JOIN
       career_summaries ON players.personId = career_summaries.personId
 WHERE career_summaries.personID = (
                                       SELECT personID
                                         FROM career_summaries
                                        WHERE blocks = (
                                                           SELECT MAX(blocks)
                                                             FROM career_summaries
                                                       )
                                   )
UNION
SELECT players.firstname,
       players.lastname,
       'steals' AS category,
       career_summaries.steals AS value
  FROM players
       JOIN
       career_summaries ON players.personId = career_summaries.personId
 WHERE career_summaries.personID = (
                                       SELECT personID
                                         FROM career_summaries
                                        WHERE steals = (
                                                           SELECT MAX(steals)
                                                             FROM career_summaries
                                                       )
                                   )
UNION
SELECT players.firstname,
       players.lastname,
       'assists' AS category,
       career_summaries.assists AS value
  FROM players
       JOIN
       career_summaries ON players.personId = career_summaries.personId
 WHERE career_summaries.personID = (
                                       SELECT personID
                                         FROM career_summaries
                                        WHERE assists = (
                                                            SELECT MAX(assists)
                                                              FROM career_summaries
                                                        )
                                   );
-- SQL 查詢語法終點
"""

## 57. 從 `nba` 資料庫查詢截至 2021-03-31 各球隊陣中場均得分大於等於 20 分（`ppg >= 20`）的球員人數，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(30, 2) 的查詢結果。

```
                 team_name  number_of_players
0            Brooklyn Nets                  4
1       Los Angeles Lakers                  2
2   Portland Trail Blazers                  2
3       Washington Wizards                  2
4            Atlanta Hawks                  1
5         Dallas Mavericks                  1
6    Golden State Warriors                  1
7              LA Clippers                  1
8          Milwaukee Bucks                  1
9   Minnesota Timberwolves                  1
10    New Orleans Pelicans                  1
11      Philadelphia 76ers                  1
12            Phoenix Suns                  1
13       San Antonio Spurs                  1
14               Utah Jazz                  1
15          Boston Celtics                  0
16       Charlotte Hornets                  0
17           Chicago Bulls                  0
18     Cleveland Cavaliers                  0
19          Denver Nuggets                  0
20         Detroit Pistons                  0
21         Houston Rockets                  0
22          Indiana Pacers                  0
23       Memphis Grizzlies                  0
24              Miami Heat                  0
25         New York Knicks                  0
26   Oklahoma City Thunder                  0
27           Orlando Magic                  0
28        Sacramento Kings                  0
29         Toronto Raptors                  0
```

In [87]:
find_number_of_top_scorers_by_teams_from_nba =\
"""
-- SQL 查詢語法起點
SELECT teams.fullName AS team_name,
       IFNULL(top_scorers_by_teams.number_of_players,0) AS number_of_players
  FROM teams
  LEFT JOIN(SELECT teams.fullName,
       COUNT(*) AS number_of_players
             FROM career_summaries
             JOIN players
               ON career_summaries.personId = players.personId
             JOIN teams
               ON players.teamId = teams.teamId
            WHERE career_summaries.ppg >= 20
            GROUP BY teams.fullName) AS top_scorers_by_teams
   ON teams.fullName = top_scorers_by_teams.fullName
ORDER BY number_of_players DESC,
         team_name;
-- SQL 查詢語法終點
"""

## 58. 從 `twElection2020` 資料庫查詢中國國民黨與民主進步黨在 2020 年選舉的得票率，包含總統副總統、不分區立委與區域立委，參考下列的預期查詢結果。

PS 不分區立委的投票資料記錄於資料表 `legislative_at_large`，區域立委的投票資料記錄於資料表 `legislative_regional`。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(2, 4) 的查詢結果。

```
   party presidential legislative_regional legislative_at_large
0  中國國民黨       38.61%               40.71%               33.36%
1  民主進步黨       57.13%               45.11%               33.98%
```

In [88]:
summarize_kmt_dpp_votes_from_twelection2020 =\
"""
-- SQL 查詢語法起點

-- SQL 查詢語法終點
"""

## 59. 從 `twElection2020` 資料庫查詢代表中國國民黨參選總統副總統的韓國瑜/張善政組合，在台灣 7,737 個選舉區（以村鄰里為一個選舉區）贏得的選舉區（得票數大於 `>` 蔡英文/賴清德組合）以及淨贏得票數，參考下列的預期查詢結果。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(1332, 4) 的查詢結果。

```
     county town village  net_winning_votes
0       金門縣  金城鎮     西門里               2190
1       高雄市  左營區     海勝里               2096
2       臺北市  松山區     自強里               1802
3       桃園市  中壢區     自立里               1763
4       桃園市  龜山區     陸光里               1560
...     ...  ...     ...                ...
1327    基隆市  中山區     居仁里                  1
1328    彰化縣  田中鎮     碧峰里                  1
1329    新竹縣  湖口鄉     仁勢村                  1
1330    澎湖縣  馬公市     東衛里                  1
1331    臺中市  太平區     勤益里                  1

[1332 rows x 4 columns]
```

In [89]:
find_kmt_winning_villages_from_twelection2020 =\
"""
-- SQL 查詢語法起點

-- SQL 查詢語法終點
"""

## 執行測試！

Runtime -> Restart and run all -> Yes -> Run anyway.

In [90]:
class TestMoreMoreExercises(unittest.TestCase):
    def test_51_summarize_confirmed_by_us_states_from_covid19(self):
        confirmed_by_us_states_from_covid19 = pd.read_sql(summarize_confirmed_by_us_states_from_covid19, conn)
        self.assertEqual(confirmed_by_us_states_from_covid19.shape, (10, 2))
        column_values = set(confirmed_by_us_states_from_covid19.iloc[:, 0].values)
        self.assertTrue('California' in column_values)
        self.assertTrue('Texas' in column_values)
        self.assertTrue('Florida' in column_values)
        self.assertTrue('New York' in column_values)
        self.assertTrue('Illinois' in column_values)
    def test_52_summarize_confirmed_deaths_of_east_asia_from_covid19(self):
        confirmed_deaths_of_east_asia_from_covid19 = pd.read_sql(summarize_confirmed_deaths_of_east_asia_from_covid19, conn)
        self.assertEqual(confirmed_deaths_of_east_asia_from_covid19.shape, (5, 3))
        column_values = set(confirmed_deaths_of_east_asia_from_covid19.iloc[:, 0].values)
        self.assertTrue('China' in column_values)
        self.assertTrue('Japan' in column_values)
        self.assertTrue('Korea, South' in column_values)
        self.assertTrue('Singapore' in column_values)
        self.assertTrue('Taiwan' in column_values)
    def test_53_find_most_frequent_directors_from_imdb(self):
        most_frequent_directors_from_imdb = pd.read_sql(find_most_frequent_directors_from_imdb, conn)
        self.assertEqual(most_frequent_directors_from_imdb.shape, (3, 2))
        column_values = set(most_frequent_directors_from_imdb.iloc[:, 0].values)
        self.assertTrue('Christopher Nolan' in column_values)
        self.assertTrue('Martin Scorsese' in column_values)
        self.assertTrue('Stanley Kubrick' in column_values)
        column_values = set(most_frequent_directors_from_imdb.iloc[:, 1].values)
        self.assertTrue(7 in column_values)
    def test_54_find_most_frequent_actors_from_imdb(self):
        most_frequent_actors_from_imdb = pd.read_sql(find_most_frequent_actors_from_imdb, conn)
        self.assertEqual(most_frequent_actors_from_imdb.shape, (1, 3))
        column_values = set(most_frequent_actors_from_imdb.iloc[:, 1].values)
        self.assertTrue('Robert De Niro' in column_values)
        column_values = set(most_frequent_actors_from_imdb.iloc[:, 2].values)
        self.assertTrue(9 in column_values)
    def test_55_find_lead_actors_from_imdb(self):
        lead_actors_from_imdb = pd.read_sql(find_lead_actors_from_imdb, conn)
        self.assertEqual(lead_actors_from_imdb.shape, (13, 3))
        titles = set(lead_actors_from_imdb.iloc[:, 0].values)
        self.assertTrue('The Shawshank Redemption' in titles)
        self.assertTrue('The Godfather' in titles)
        self.assertTrue('Fight Club' in titles)
        self.assertTrue('Forrest Gump' in titles)
        self.assertTrue('Inception' in titles)
        lead_actors = set(lead_actors_from_imdb.iloc[:, 2].values)
        self.assertTrue('Tim Robbins' in lead_actors)
        self.assertTrue('Marlon Brando' in lead_actors)
        self.assertTrue('Leonardo DiCaprio' in lead_actors)
    def test_56_find_max_stats_total_from_nba(self):
        max_stats_total_from_nba = pd.read_sql(find_max_stats_total_from_nba, conn)
        self.assertEqual(max_stats_total_from_nba.shape, (5, 4))
        first_names = set(max_stats_total_from_nba.iloc[:, 0].values)
        self.assertTrue('Chris' in first_names)
        self.assertTrue('Dwight' in first_names)
        self.assertTrue('LeBron' in first_names)
    def test_57_find_number_of_top_scorers_by_teams_from_nba(self):
        number_of_top_scorers_by_teams_from_nba = pd.read_sql(find_number_of_top_scorers_by_teams_from_nba, conn)
        self.assertEqual(number_of_top_scorers_by_teams_from_nba.shape, (30, 2))
        column_values = number_of_top_scorers_by_teams_from_nba.iloc[:, 1].values
        self.assertEqual(np.max(column_values), 4)
        self.assertEqual(np.min(column_values), 0)
    def test_58_summarize_kmt_dpp_votes_from_twelection2020(self):
        kmt_dpp_votes_from_twelection2020 = pd.read_sql(summarize_kmt_dpp_votes_from_twelection2020, conn)
        self.assertEqual(kmt_dpp_votes_from_twelection2020.shape, (2, 4))
        column_values = set(kmt_dpp_votes_from_twelection2020.iloc[:, 0].values)
        self.assertTrue('中國國民黨' in column_values)
        self.assertTrue('民主進步黨' in column_values)
    def test_59_find_kmt_winning_villages_from_twelection2020(self):
        kmt_winning_villages_from_twelection2020 = pd.read_sql(find_kmt_winning_villages_from_twelection2020, conn)
        self.assertEqual(kmt_winning_villages_from_twelection2020.shape, (1332, 4))

suite = unittest.TestLoader().loadTestsFromTestCase(TestMoreMoreExercises)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)
#cwd = os.getcwd()
folder_name = "15-more-more-exercises"
with open("exercise_index.json", "r") as content:
    exercise_index = json.load(content)
chapter_name = exercise_index[folder_name]

test_51_summarize_confirmed_by_us_states_from_covid19 (__main__.TestMoreMoreExercises.test_51_summarize_confirmed_by_us_states_from_covid19) ... ok
test_52_summarize_confirmed_deaths_of_east_asia_from_covid19 (__main__.TestMoreMoreExercises.test_52_summarize_confirmed_deaths_of_east_asia_from_covid19) ... ok
test_53_find_most_frequent_directors_from_imdb (__main__.TestMoreMoreExercises.test_53_find_most_frequent_directors_from_imdb) ... ok
test_54_find_most_frequent_actors_from_imdb (__main__.TestMoreMoreExercises.test_54_find_most_frequent_actors_from_imdb) ... ok
test_55_find_lead_actors_from_imdb (__main__.TestMoreMoreExercises.test_55_find_lead_actors_from_imdb) ... ok
test_56_find_max_stats_total_from_nba (__main__.TestMoreMoreExercises.test_56_find_max_stats_total_from_nba) ... ok
test_57_find_number_of_top_scorers_by_teams_from_nba (__main__.TestMoreMoreExercises.test_57_find_number_of_top_scorers_by_teams_from_nba) ... ok
test_58_summarize_kmt_dpp_votes_from_twelection2020 (__m

In [91]:
print("您在「{}」章節中的 {} 道 SQL 練習答對了 {} 題。".format(chapter_name, number_of_test_runs, number_of_successes))

您在「更多綜合練習題」章節中的 9 道 SQL 練習答對了 7 題。
